## Imports and Params

In [17]:
from newsapi import NewsApiClient
from nytimesarticle import articleAPI
import pandas_datareader.data as web
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dateutil
import time
import quandl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pandas.plotting import autocorrelation_plot
from pandas.tools.plotting import lag_plot
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit


In [18]:
# Params
# extra API keys, comment out as necessary
#api = articleAPI("b23351c6f9314694bfe4f4929a2b72c5") 
#api = articleAPI("787bd4db8e704bbf9cebe8b7941827e0") 
#api = articleAPI("f8b402f42ed14b249fd5accc95a050dd") 
#api = articleAPI("c91a676aeaef40fd844409c8b0bef485")
#api = articleAPI("c43133d654134109868299ff505e7c55")
#api = articleAPI("eb427ebc2336423ead4d350cfa4e900b")
#api = articleAPI("b538de93f1a9459da22b150d7b53cb6f")
api = articleAPI("88f587ed149d4478b4490168d61ed9dc")

quandl.ApiConfig.api_key = "2S7d7eeL5VZrLup9pKg5"
end_date = (datetime.datetime.now() - datetime.timedelta(days=3)).isoformat()
start_date = (datetime.datetime.now() - datetime.timedelta(days=2*365)).isoformat()
left_sources = 'The New York Times'
right_sources = 'Fox News'
center_sources = 'Reuters AP The Wall Street Journal'
all_sources = left_sources + ' ' + right_sources + ' ' + center_sources
replace_list = ['Corp', 'Inc.', 'Inc', '.com', 'plc', ',', 'Co.']
# domain

In [42]:
def main():
    # retrieve s&p 500 companies 
    df = pd.read_csv('constituents_csv.csv')
    companies = df['Name']
    company_symb = {}

    # Iterate through companies
    # for company in companies:
    for i in range(30,31):

        company = companies.loc[i]

        # get company ticker
        company_symb[company] = df[df['Name'] == company]['Symbol']
        ticker = company_symb[company].values[0]
        
        # get rid of suffixes from company name
        for word in replace_list:
            company = company.replace(word, '')

        # Read in data
        df_master = read_in_stock_data(ticker, start_date, end_date)
        
        # Calculate 25 lags first
        for i in range(25):
            df_master['X_t-' + str(i+1)] = df_master['X_t'].shift(i+1)
            
        # Remove the ith data point because of shift
        df_master = df_master.iloc[i+1:]
        
        ###### Add in sentiment values ######
        
        # Query news articles
        trading_dates = df_master.index
        dict_master = query_news_articles(company, start_date, end_date, trading_dates, sources=all_sources)
        
        # Add sentiment columns in dataframe
        df_master['Pos_t-1'] = 0
        df_master['Neu_t-1'] = 0
        df_master['Neg_t-1'] = 0

        # iterate through dates
        for date in dict_master.keys():
            # when you enter seniment into the dataframe, use the before date not after
            average_sentiment_dict = calculate_sentiment(dict_master[date])

            # Plug this into df
            df_master.at[date,'Pos_t-1'] = average_sentiment_dict['pos']
            df_master.at[date,'Neu_t-1'] = average_sentiment_dict['neu']
            df_master.at[date,'Neg_t-1'] = average_sentiment_dict['neg']
         
        ###### Per Fold do cross validation ######
        MSE_list_AR, coef_list_AR = cross_validate_AR(df_master)   
        print('Mean Squared Error List:', MSE_list_AR)
        print('Coefficient list:',coef_list_AR)
        MSE_list_ADL, coef_list_ADL = cross_validate_ADL(df_master)
        print('Mean Squared Error List:', MSE_list_ADL)
        print('Coefficient list:',coef_list_ADL)
            
        print(i, ticker, 'success')
        #df_current.to_csv('./data/'+ticker+'.csv')
        #df_current = df_current.reset_index()
        return df_master

            # print(i, ticker, 'failed')
            # pass

In [51]:
a = main()

page 0
2018-05-03T18:54:19+0000
2018-01-25T18:11:56+0000
2018-04-27T04:01:41+0000
2018-04-04T09:11:09+0000
2018-03-29T10:40:59+0000
2018-05-03T09:45:05+0000
2018-04-18T23:16:31+0000
2018-03-15T23:20:22+0000
2018-01-30T12:22:10+0000
2017-10-26T21:45:11+0000
page 1
2018-01-28T19:30:49+0000
2018-03-29T15:46:12+0000
2018-03-23T17:10:19+0000
2018-04-06T12:00:01+0000
2017-12-21T20:23:49+0000
2017-11-16T21:08:18+0000
2017-07-20T09:00:38+0000
2017-08-24T14:19:01+0000
2017-03-27T16:01:10+0000
2017-08-17T00:10:18+0000
page 2
2017-05-01T18:37:35+0000
2017-07-13T14:21:16+0000
2017-06-29T13:30:17+0000
2017-03-31T18:20:20+0000
2018-03-19T22:08:41+0000
2018-04-24T14:00:07+0000
2017-05-19T11:00:33+0000
2018-03-16T09:57:09+0000
2018-03-04T16:34:40+0000
2018-04-20T09:53:03+0000
page 3
2017-09-15T14:21:28+0000
2018-04-16T13:16:28+0000
2017-04-06T16:16:01+0000
2017-03-30T18:24:46+0000
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19


In [40]:
def cross_validate_AR(df):
    """ Runs the backtest with k fold cross validation for AR model
    Params:
        df (pd.Dataframe): Dataframe with X_t, lags, and sentiment score
    Returns:
        MSE_list (list): List of MSE per fold of cross-validation
        coef_list (list): List of coef per fold of cross-validation
    """
    tscv = TimeSeriesSplit(n_splits=8)
    X = df.loc[:, df.columns !='X_t']
    Y = df['X_t']
    
    MSE_list = []
    coef_list = []
        
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        Y_train, Y_test = Y.iloc[train_index], Y[test_index]
        
        # find optimal lag period
        lag = get_lag_period(Y_train)

        # filter the lags and sentiment values
        X_train = X_train.iloc[:,range(lag-1)]
        X_test = X_test.iloc[:,range(lag-1)]

        MSE, coef = fit_and_run_regression(X_train, Y_train, X_test, Y_test)
        MSE_list.append(MSE)
        coef_list.append(coef)
        
    return MSE_list, coef_list


In [50]:
def cross_validate_ADL(df):
    """ Runs the backtest with k fold cross validation for ADL model
    Params:
        df (pd.Dataframe): Dataframe with X_t, lags, and sentiment score
    Returns:
        MSE_list (list): List of MSE per fold of cross-validation
        coef_list (list): List of coef per fold of cross-validation
    """
    tscv = TimeSeriesSplit(n_splits=8)
    X = df.loc[:, df.columns !='X_t']
    Y = df['X_t']
    
    MSE_list = []
    coef_list = []
        
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        Y_train, Y_test = Y.iloc[train_index], Y[test_index]
        
        # find optimal lag period
        lag = get_lag_period(Y_train)

        # filter the lags and sentiment values
        X_train = X_train.iloc[:,range(lag-1)]
        X_test = X_test.iloc[:,range(lag-1)]
        
        # Add in the sentiment values
        X_train['Pos_t-1'] = df.iloc[train_index,-3]
        X_train['Neu_t-1'] = df.iloc[train_index,-2]
        X_train['Neg_t-1'] = df.iloc[train_index,-1]
        
        X_test['Pos_t-1'] = df.iloc[test_index,-3]
        X_test['Neu_t-1'] = df.iloc[test_index,-2]
        X_test['Neg_t-1'] = df.iloc[test_index,-1]
        
        X_train = X_train.fillna(0)
        X_test = X_test.fillna(0)

        MSE, coef = fit_and_run_regression(X_train, Y_train, X_test, Y_test)
        MSE_list.append(MSE)
        coef_list.append(coef)
        
    return MSE_list, coef_list


### Methods Used for Data Collection

In [5]:
def calculate_sentiment(sentence_arr):
    """ Returns the average sentiment of the array
    Params:
        sentence_arr(Array): Array of setences that we have to calculate
        the sentiment of.
    Returns:
        sentiment (dictionary): Takes the average of all sentences
        format of score is {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
    """
    sia = SentimentIntensityAnalyzer()
    df = df = pd.DataFrame(columns=['neg','neu','pos','compound'])
    for sentence in sentence_arr:
        sentiment = sia.polarity_scores(sentence)
        df_sentiment = pd.DataFrame([sentiment], columns=sentiment.keys())
        df = df.append(df_sentiment)
    
    avg_sentiment = dict(df.mean())    
    return avg_sentiment 

In [6]:
def query_news_articles(company, start_date, end_date, trading_dates, sources):
    """ Queries news article for a certain time frame and split it by dates
        Note that
    Params:
        company (String): Name of company
        start_date (String): Start date in format of "2001-12-31"
        end_date (String): End date in format of "2001-12-31"
         trading_dates (Array of Strings): Array of dates when the market was open
                 dates in format of "2001-12-31"
        sources (Array of Strings): Array of different news sources
    Returns:
        company_dic (dictionary): keys are date, values are array of headlines
    """
    company_dict = {k: [] for k in trading_dates.date}
    start_date = int(start_date.replace("-","").split('T')[0])
    ending_date = (dateutil.parser.parse(end_date)-datetime.timedelta(days=1)).isoformat()
    end_date = int(ending_date.replace("-","").split('T')[0])
    newsdata = api.search(q=company, begin_date = start_date,
                           end_date = end_date,
                          fq='headline:('+company+ ') OR body:('+company+') AND source:(' + sources + ')',
                          page = 0,
                          facet_filter = True)
                               

    #print(newsdata) # newsdata is full HTTP response
    number_of_hits = newsdata['response']['meta']['hits']
    number_of_pages = (number_of_hits // 10) + 1
    
    time.sleep(1)
    # page through results and add headlines to companY_dict
    for i in range(0, min(number_of_pages,200)):
        print('page', i)
        newsdata = api.search(q=company, begin_date = start_date,
                           end_date = end_date,
                          fq='headline:('+company+ ') OR body:('+company+') AND source:(' + sources + ')',
                          page = i,
                          facet_filter = True)
        articles = newsdata['response']['docs']
        for article in articles:
            relevance = article['score']
            if relevance >= 0.005: 
                headline = article['headline']['main']
                blurb = article['snippet']
                # print(article['pub_date'], '\t', article['headline']['main'])
            
                # description = article['description']
                # format of date is 2018-04-13T00:46:59Z (UTC format)
                publish_date = article['pub_date'] 
                print(publish_date)
                # adjust date for trading day
                publish_date, publish_time = publish_date.split('T')
                date_arr = publish_date.split('-')
                publish_datetime = datetime.date(int(date_arr[0]), int(date_arr[1]), int(date_arr[2]))
                time_arr = publish_time[:-1].split(':')
                # stock market closes at 4:00 PM EST; if article published after 
                # 16:00:00+4:00:00 = 20:00:00 UTC headline affects next trading day;
                # otherwise affects current trading day
                trading_datetime = publish_datetime
                if int(time_arr[0]) >= 20:
                    trading_datetime += datetime.timedelta(days=1)
                
                # if given trading_date invalid (ie if article published on Friday 
                # after market close, Saturday, or Sunday before 4 pm est) push trading_date
                # to the following Monday (ie first valid trading_date)
                while trading_datetime not in trading_dates:
                    trading_datetime += datetime.timedelta(1)
                company_dict[trading_datetime].append(headline)
                # company_dict[trading_datetime].append(blurb) include 'snippet' in sentiment analysis
        time.sleep(1)
        
    return company_dict

In [7]:
def get_lag_period(X_train):
    """ Finds the optimal lab period given a ticker and the start date and end date
    Params:
        X_train (df): Training data used to calculate lag back
    Returns:
        lag (int): Number of lag periods
    """
    model = AR(X_train)
    model_fit = model.fit()
    lag = model_fit.k_ar
    
    return lag
    

In [38]:
def read_in_stock_data(ticker, start_date, end_date):
    """ Initializes a data frame for a certain ticker
    Params:
        ticker (String): Stock ticker to be analyzed
        start_date (String): Start date in format of "2001-12-31"
        end_date (String): End date in format of "2001-12-31"
        lag (int): Number of lag periods
    Returns:
        dataframe (pd.Dataframe): Dataframe with index 'Date' and column 'X_t'
    """
    # Get the data in a dataframe
    dataframe = web.DataReader(ticker, 'morningstar', start_date, end_date)['Close']
    dataframe = pd.Series.to_frame(dataframe)
    dataframe.reset_index(level=0, drop=True, inplace=True)
    dataframe.columns = ['X_t']
    return dataframe

In [37]:
def fit_and_run_regression(X_train, Y_train, X_test, Y_test, doprint=False):
    """ Fits the model with train and test data
    Params:
        X_train (pd.Dataframe): X training data
        Y_train (pd.Dataframe): Y training data
    Returns:
        MSE (float): Mean Squared Error 
    """
    LinReg = LinearRegression(normalize=True)
    LinReg.fit(X_train,Y_train)
    Y_pred = LinReg.predict(X_test)
    MSE = mean_squared_error(Y_test, Y_pred)
    coefficients = LinReg.coef_
    if doprint:
        print("R^2 Value: %.2f" %LinReg.score(X_test, Y_test))
        print("Mean squared error: %.2f "% MSE)
        print("Coefficients for the Regression are: " ,coefficients)
    
    return MSE, coefficients

### Try AR Regression without the package

In [12]:
# Try normal linear regression using lag period lags
lag = get_lag_period('AMZN', start_date, end_date)
print('Number of lags:',lag)
df_test = initialize_dataframe('AMZN', start_date, end_date, lag)
df_test.head()

Y = df_test['X_t']
X = df_test.loc[:, df_test.columns !='X_t']
X_train, X_test = X[:int(len(X)*(0.8))], X[int(len(X)*(0.8)):]
Y_train, Y_test = Y[:int(len(Y)*(0.8))], Y[int(len(Y)*(0.8)):]

LinReg = LinearRegression(normalize=True)
LinReg.fit(X_train,Y_train)
Y_pred = LinReg.predict(X_test)
print("R^2 Value: %.2f" %LinReg.score(X_test, Y_test))
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, Y_pred))
print("Coefficients for the Regression are: " ,LinReg.coef_)

TypeError: get_lag_period() takes 1 positional argument but 3 were given

In [13]:
Y_plot = Y_test.copy(deep=True)
Y_plot = Y_plot.to_frame()
Y_plot['Predicted'] = pd.Series(Y_pred, index=Y_plot.index)
Y_plot.columns = ['Actual', 'Predicted']
print(Y_plot.head())

plt.figure(figsize=(10,10))
plt.title('Predicted Vs. Actual Plot AR model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.plot(Y_plot.index, Y_plot['Actual'], label='Actual')
plt.plot(Y_plot.index, Y_plot['Predicted'],label='Predicted')
plt.legend()

plt.show()

NameError: name 'Y_test' is not defined

### Try Regression with sentiment analysis

In [ ]:
df_sentiment = main()
df_sentiment.head()

In [ ]:
# Do the regression sentiment

# Count number of nulls and replace them with 0's
df_sentiment.isnull().sum()
df_sentiment = df_sentiment.fillna(0)
df_sentiment.head()

Y = df_sentiment['X_t']
X = df_sentiment.loc[:, df_sentiment.columns !='X_t']
X_train, X_test = X[:int(len(X)*(0.8))], X[int(len(X)*(0.8)):]
Y_train, Y_test = Y[:int(len(Y)*(0.8))], Y[int(len(Y)*(0.8)):]

LinReg_sentiment = LinearRegression(normalize=True)
LinReg_sentiment.fit(X_train,Y_train)
Y_pred = LinReg_sentiment.predict(X_test)
print("R^2 Value: %.2f" %LinReg_sentiment.score(X_test, Y_test))
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, Y_pred))
print("Coefficients for the Regression are: " , LinReg_sentiment.coef_)

In [ ]:
# Plot
Y_plot = Y_test.copy(deep=True)
Y_plot = Y_plot.to_frame()
Y_plot['Predicted'] = pd.Series(Y_pred, index=Y_plot.index)
Y_plot.columns = ['Actual', 'Predicted']
print(Y_plot.head())

plt.figure(figsize=(10,10))
plt.title('Predicted Vs. Actual Plot ADL Model')
plt.xlabel('Date')
plt.ylabel('Price')
plt.plot(Y_plot.index, Y_plot['Actual'], label='Actual')
plt.plot(Y_plot.index, Y_plot['Predicted'],label='Predicted')
plt.legend()

plt.show()

## Regression

In [ ]:
# Run to get amazon's df
df_current = main()
df_current.head()


In [ ]:
# Find index of earlier data
df_current.loc[datetime.datetime(2018,2,7)]
# df_current.loc[df_current['Date'] == datetime.datetime(2018,2,7)]

In [ ]:
# Use this to splice the dataframe to 2018-02-07 and after
df_changed = df_current[456:].fillna(0)
df_changed.head()

In [ ]:
# Plot autocorrelation values
plt.figure()
autocorrelation_plot(df_changed['X_t'])
plt.show()

In [ ]:
# Split into train and test
X = df_changed['X_t']
train, test = X[:int(len(X)*(0.8))], X[int(len(X)*(0.8)):]
# Fit the model
model = AR(train)
model_fit = model.fit()
print('Lag: %s' % model_fit.k_ar)
print('Coefficients: %s' % model_fit.params)

In [ ]:
# make predictions
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
for i in range(len(predictions)):
    print('predicted=%f, expected=%f' % (predictions[i], test[i]))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
R2 = r2_score(test, predictions)
print('Test R^2 Score: %.3f ' % R2)

In [ ]:
# Plot the preditions
plt.figure(figsize=(10,7))
plt.plot(test, label='Actual Values')
plt.title('Predicted vs. Actual Values')
plt.xlabel('Date')
plt.ylabel('Price')
plt.plot(predictions, label='Predicted Values')
plt.legend()
plt.show()